In [1]:
!pip install torch scikit-learn pandas numpy
!git clone https://github.com/ArzeonXyl/spaced-rl-classifier.git
%cd spaced-rl-classifier
!pip install -e .

# ===============================
# Test SpacedRLClassifier dengan Iris dataset
# ===============================
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from spaced_rl_classifier import SpacedRLClassifier, SimpleMLP, RLSchedulerPolicy

# -------------------------------
# Load & preprocess dataset
# -------------------------------
iris = load_iris()
X = iris.data.astype('float32')
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# -------------------------------
# Build items pakai helper
# -------------------------------
items = SpacedRLClassifier.build_items(X_train, y_train)
items += SpacedRLClassifier.build_items(X_test, y_test)

train_count = len(y_train)
n_features = X.shape[1]
n_classes = len(set(y))

# -------------------------------
# Setup model & policy
# -------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SimpleMLP(dim=n_features, hidden=64, n_classes=n_classes)
policy = RLSchedulerPolicy(n_classes=n_classes, emb_dim=8, hidden=32)

clf = SpacedRLClassifier(model=model, policy=policy, device=device)

# -------------------------------
# Fit model (atur epochs & batch_size sesuai kebutuhan)
# -------------------------------
clf.fit(items, train_count=train_count, epochs=5, batch_size=12, k_retrieval=2)

# -------------------------------
# Evaluate
# -------------------------------
test_idx = list(range(train_count, len(items)))
res = clf.evaluate(test_idx)

print("Test Accuracy:", res["accuracy"])
print(res["report"])


Cloning into 'spaced-rl-classifier'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 27 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 35.41 KiB | 2.95 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/spaced-rl-classifier
Obtaining file:///content/spaced-rl-classifier
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for spaced_rl_classifier (pyproject.toml) ... done
  Created wheel for spaced_rl_classifier: filename=spaced_rl_classifier-0.1.0-0.editable-py3-none-any.whl size=2969 sha256=92eeea3ea563a4252f13e032a4458a75f157e32a416e42ce58a5b4244a6bcf18
  Stored in directory: /tmp/pip-ephem-wheel-cache-yy90c_bz/wheels/29/10/6a/b9f2f0e37f3062a4d466407bbad3502ddfbfc3